In [3]:
from gql import Client, gql
from gql.transport.requests import RequestsHTTPTransport
import numpy as np
import pandas as pd

In [5]:
ACCESS_TOKEN = input("Enter your Yelp API access token: ")
HEADERS = {"Authorization": "Bearer " + ACCESS_TOKEN}
TRANSPORT = RequestsHTTPTransport(
    url="https://api.yelp.com/v3/graphql", headers=HEADERS)
CLIENT = Client(transport=TRANSPORT, fetch_schema_from_transport=True)

In [6]:
def get_restaurants(categories, location, radius):
    query = gql(
        """
      {
        search(categories: "%s",
              location: "%s"
              radius: %d) {
          total
        }
      }
  """
        % (categories, location, np.minimum(radius, 40_000)))

    return CLIENT.execute(query)

In [24]:
df = pd.read_csv("../cities.csv", index_col=[0, 1])

data = pd.DataFrame(index=df.index)
data["population"] = df["2023 estimate"]
data["area"] = pd.to_numeric(df["land area"], errors="coerce")

# convert from sq mi to sq m, a = pi*r^2
data["radius"] = np.sqrt(data["area"]*2589988.1103/np.pi)

data.head()

,,population,area,radius
city,state,,,
New York,NY,8258035,300.5,15739.690454
Los Angeles,CA,3820914,469.5,19673.958328
Chicago,IL,2664452,227.7,13701.100885
Houston,TX,2314157,640.4,22977.332585
Phoenix,AZ,1650070,518.0,20665.162692


In [25]:
data["restaurants"] = 0

for i, row in data.iterrows():
    location = "%s, %s" % (i[0], i[1])
    # print(location)
    try:
        result = get_restaurants("vegan", location, row["radius"])
        data.loc[i, "restaurants"] = int(result["search"]["total"])
    except Exception as e:
        print(e)

In [28]:
vegan = pd.read_csv("vegan.csv", index_col=[0, 1])

data["per_100k"] = data["restaurants"] / data["population"] * 100_000

chicago = data.loc[("Chicago", "IL"), "per_100k"]
data["score"] = np.log2(data["per_100k"] / chicago)

data.sort_values("score", ascending=False, inplace=True)

/home/dan/danwahl/schelling-out/env/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [29]:
print(data.to_string())
data.to_csv("vegan.csv")

                        population   area        radius  restaurants   per_100k     score
city             state                                                                   
Atlanta          GA         510823  135.3  10561.432972           94  18.401677  2.478101
Hollywood        FL         153859   27.3   4744.115703           28  18.198480  2.462081
Portland         OR         630498  133.5  10490.944312          112  17.763736  2.427198
Berkeley         CA         118962   10.4   2928.131782           21  17.652696  2.418152
Bend             OR         104557   33.6   5263.123823           14  13.389826  2.019401
Salt Lake City   UT         209593  110.3   9535.900372           27  12.882110  1.963632
Fort Lauderdale  FL         184255   34.6   5340.869891           23  12.482701  1.918194
Burbank          CA         102755   17.3   3776.565317           11  10.705075  1.696558
Inglewood        CA         102865    9.1   2739.016478           10   9.721480  1.557511
Long Beach

In [22]:
pd.to_numeric(data["population"], errors="coerce")

city         state
New York     NY      NaN
Los Angeles  CA      NaN
Chicago      IL      NaN
Houston      TX      NaN
Phoenix      AZ      NaN
                      ..
Yuma         AZ      NaN
New Bedford  MA      NaN
Suffolk      VA      NaN
Hesperia     CA      NaN
Davenport    IA      NaN
Name: population, Length: 336, dtype: float64

In [23]:
s = data["restaurants"]